In [ ]:
import ast
import nltk
import numpy as np
import pandas as pd
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/tmdb/tmdb-movie-metadata/versions/2


In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_movies.csv'

In [ ]:
movies.head(1)


In [ ]:
credits.head(1)


In [ ]:
movies = movies.merge(credits, on = 'title')


In [ ]:
movies.head(1)


In [ ]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]


In [ ]:
movies.head(1)


In [ ]:
movies.isnull().sum()


In [ ]:
movies.dropna(inplace = True)


In [ ]:
movies.isnull().sum()


In [ ]:
movies.duplicated().sum()


In [ ]:
movies.iloc[0].genres


In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)


In [ ]:
movies.head(1)


In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)


In [ ]:
movies.head(1)


In [ ]:
def convert_cast(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 3:
            L.append(i['name'])
            count = count + 1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert_cast)


In [ ]:
movies.head(1)


In [ ]:
def get_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(get_director)


In [ ]:
movies.head(1)


In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())


In [ ]:
movies.head(1)


In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])


In [ ]:
movies.head(1)


In [ ]:
movies['tag'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']


In [ ]:
movies.head(1)


In [ ]:
new_df = movies[['movie_id', 'title', 'tag']]


In [ ]:
new_df.head()


In [ ]:
new_df['tag'] = new_df['tag'].apply(lambda x : " ".join(x))


In [ ]:
new_df.head(1)


In [ ]:
new_df['tag'] = new_df['tag'].apply(lambda x : x.lower())


In [ ]:
new_df.head(1)


In [ ]:
new_df['tag'][0]


In [ ]:
new_df['tag'][1]


In [ ]:
cv = CountVectorizer(max_features = 5000, stop_words = 'english')


In [ ]:
cv

In [ ]:
CountVectorizer(max_features=5000, stop_words='english')


In [ ]:
vector = cv.fit_transform(new_df['tag']).toarray()


In [ ]:
vector


In [ ]:
cv.get_feature_names_out()


In [ ]:
ps = PorterStemmer()


In [ ]:
def stem(text):
    L = []
    for i in text.split():
        L.append(ps.stem(i))
    return " ".join(L)


In [ ]:
new_df['tag'] = new_df['tag'].apply(stem)


In [ ]:
new_df.head(1)


In [ ]:
cv.get_feature_names_out()


In [ ]:
similarity = cosine_similarity(vector)


In [ ]:
similarity[0]


In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x: x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('John Carter')


In [ ]:
recommend('Mission to Mars')


In [ ]:
recommend('Avatar')


In [ ]:
recommend('Titanic')
